# Table of Contents
 <p>

In [ ]:
%load_ext autotime 
import sklearn as sk
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import qgrid
import os
from IPython.display import display
# %store -r
import gc

In [ ]:
print ('List of dataset in directory:')
os.listdir(datadir)
print ('Loading \'properties_2016.csv\'... ')
properties_2016 = pd.read_csv(datadir + "/properties_2016.csv")
print ('Loading \'properties_2017.csv\'... ')
properties_2017 = pd.read_csv(datadir + "/properties_2017.csv")
print ('Loading \'train_2016_v2.csv\'... ')
train_2016 = pd.read_csv(datadir + "/train_2016_v2.csv")
print ('Loading \'train_2017.csv\'... ')
train_2017 = pd.read_csv(datadir + "/train_2017.csv")
print ('Loading \'sample_submission.csv\'... ')
submission = pd.read_csv(datadir + "/sample_submission.csv")

In [ ]:
def freq_trans(series):
    """
    Return frequency of each element in a series (same index as original series)
    """
    return series.fillna('999').groupby(series.fillna('999')).transform('count') / len(series);
def high_cardinality(df, how, threshold = 200, exception = []):
    """
    Processing high cardinality features
    Args:
    * Input: Dataframe
    * how: 'freq': change each value into its frequency
           'drop': delete the feature
           'label_encode': label encoding the features
           'label_encode_na': label encoding the features and keep NAs
    * threshold (default = 200): minimum number of different values that a feature 
    must have in order to be a high cardinality feature
    * exception (list): features that are unaffected by this function
    Output: no output. The function makes direct change to the dataframe
    """
    print ('Processing high-cardinality features:')
    for x in df.columns:
        if (df[x].dtype == 'O') and (len(df[x].unique()) > threshold) and (x not in exception):
            print (x, len(df[x].unique()))
            if how == 'freq': df[x] = freq_trans(df[x])
            elif how =='drop': del df[x]
            elif how == 'label_encode': df[x] = pd.factorize(df[x])[0]
            elif how == 'label_encode_na': df[x] = pd.factorize(df[x])[0].replace(-1,np.nan)
        else: df[x] = df[x]
    return df;
# date feature
def datefeatures(df):
    df['transactiondate'] = pd.to_datetime(df['transactiondate'])
    df['dtyear'] = df['transactiondate'].dt.year
    df['dtmonth'] = df['transactiondate'].dt.month
    df['dtday'] = df['transactiondate'].dt.day
    df['dtdayofyear'] = df['transactiondate'].dt.dayofyear
    df['dtweekofyear'] = df['transactiondate'].dt.weekofyear
    df['dtdayofweek'] = df['transactiondate'].dt.dayofweek
    df['dtquarter'] = df['transactiondate'].dt.quarter
    df = df.drop('transactiondate',axis=1)
    return df

In [ ]:
properties_2016['dtyear']= 2016
properties_2017['dtyear']= 2017
properties = pd.merge(properties_2016,properties_2017, how = 'outer')
del properties_2016,properties_2017
gc.collect()
train = pd.merge(train_2016,train_2017, how = 'outer')

In [ ]:
# Model 1
# Change feature into categorical
obj = properties.columns[properties.columns.str.contains('id|fips|censustractandblock')]
obj = obj.drop('parcelid')
print (obj)
for i in obj: properties[i] = properties[i].astype(object);
# Additional features: Number of NaNs in each row
properties['Number of NaNs'] = properties.isnull().sum(axis=1)
# Processing categorical features
properties = (properties.pipe(high_cardinality, how = 'freq', threshold = 150)
                        .pipe(pd.get_dummies,dummy_na = True))
# Additional features:
# Number of properties in the zip
zip_count = properties['regionidzip'].value_counts().to_dict()
properties['N-zip_count'] = properties['regionidzip'].map(zip_count)